In [285]:
import requests 
from bs4 import BeautifulSoup as Soup 
import re 
import pandas as pd 

In [286]:
# with open("fin_database.html", "rb") as infile: 
#     tmp = infile.read()

# tmp_soup = Soup(tmp)

In [287]:
urls = []

In [ ]:
urls = ["22-1","18","13A", "08A"]

In [288]:
url = "https://www.oes-cs.dk/bevillingslove/doctopic.cgi?book=BEVPUBL.FL13A&topic=19.22.01&searchtype=3"
tmp = requests.get(url).content 
tmp_string = str(tmp)
# tmp_soup = Soup(tmp)
# print(tmp_soup)

In [289]:
type(tmp_string)

str

In [290]:
tmp_decode = tmp.decode('iso8859_10')

In [291]:
tmp_decode = tmp_decode.replace("<B>","")
tmp_decode = tmp_decode.replace("</B>","")
tmp_list = tmp_decode.split("\r\n")

In [292]:
optage_num = 0 
virk_num = 0 
for i in tmp_list:
    if "Optagelsesskøn" in i: 
        print(tmp_list.index(i))
        optage_num = tmp_list.index(i)
    elif "Virksomhedsoversigt" in i: 
        virk_num = tmp_list.index(i)
        

112


In [293]:
optag = tmp_list[optage_num:virk_num]

virk = tmp_list[virk_num:]


In [294]:
# virk

In [295]:
optag = [i.split("|")[1:-1] for i in optag if "|" in i]
virk = [i.split("|")[1:-1] for i in virk if "|" in i]

optag[0][0] = "Finanslov"
optag[1][0] = "År"
virk[0][0] = "Finanslov"
virk[1][0] = "År"

In [296]:
# virk

In [297]:
for i in range(len(optag)):
    for j in range(len(optag[i])):
        optag[i][j] = optag[i][j].replace(".","")
        optag[i][j] = re.sub("^\s+(.+?)$",r"\1",optag[i][j])
        optag[i][j] = re.sub("\s+$",r"",optag[i][j])
        if optag[i][j] == "-":
            optag[i][j] == None 
        # optag[i][j] = optag[i][j].replace(" ","")
        try: 
            optag[i][j] = int(optag[i][j])
        except ValueError:
            pass 
        except TypeError:
            pass 
        if optag[i][j] == "achelor":
            optag[i][j] = "Bachelor"
        elif optag[i][j] == "andidat":
            optag[i][j] = "Kandidat"
        elif optag[i][j] == "alt":
            optag[i][j] = "Total"
    if optag[i][1:] == [" "] * (len(optag[i]) -1): 
        optag[i+1][0] = optag[i][0] + " " + optag[i+1][0]

In [298]:
# virk

In [299]:
act_num = 0 

for i in range(len(virk)):
    for j in range(len(virk[i])):
        virk[i][j] = virk[i][j].replace(".","")
        virk[i][j] = re.sub("^\s+(.+?)\s+$",r"\1",virk[i][j])
        virk[i][j] = re.sub("\s+$", "", virk[i][j])
        virk[i][j] = re.sub("\s+", " ",virk[i][j])
        if virk[i][j] == "-": 
            virk[i][j] = None 
        # virk[i][j] = virk[i][j].replace(" ","")
        try: 
            virk[i][j] = int(virk[i][j])
        except ValueError:
            pass 
        except TypeError:
            pass 
        if virk[i][j] == "ktiviteter": 
            virk[i][j] = "Aktiviteter"
            act_num = i 
        elif virk[i][j] == "Aktiviteter":
            act_num = i 
    if virk[i][1:] == [""] * (len(virk[i]) -1) and len(virk[i]) > 1 : 
        virk[i+1][0] = virk[i][0] + " " + virk[i+1][0]
        if virk[i][j] == "alt omsætning (mio kr)": 
            virk[i][j] = "Total omsætning (mio kr)"
        
aktivitet = virk[act_num+1:]
virk = virk[:act_num]


In [300]:
virk_dict = {}
optag_dict = {}
aktivitet_dict = {}


In [301]:
# virk

In [302]:
for i in virk: 
    if i[1:] == [""] * len(i[1:]):
        continue 
    virk_dict[i[0]] = i[1:]
for i in optag:
    if i[1:] == [""] * len(i[1:]):
        continue 
    optag_dict[i[0]] = i[1:]
for i in aktivitet:
    if i[1:] == [""] * len(i[1:]):
        continue 
    aktivitet_dict[i[0]] = i[1:]

In [303]:
virk_dict

{'Finanslov': ['R', 'R', 'R', 'R', 'R', 'B', 'F', 'BO 1', 'BO 2', 'BO 3'],
 'År': [2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016],
 'I alt omsætning (mio kr)': [6853,
  7132,
  7426,
  7777,
  7933,
  7826,
  7950,
  8084,
  8185,
  8027],
 'Tilskud i alt UI': [4669,
  4703,
  4936,
  4999,
  5097,
  4984,
  5035,
  5057,
  5043,
  4763],
 'Uddannelsestilskud fra UI': [1625,
  1640,
  1707,
  1790,
  1808,
  1789,
  1787,
  1794,
  1791,
  1720],
 'Heltidsuddannelse': [1461,
  1439,
  1474,
  1568,
  1580,
  1610,
  1660,
  1680,
  1685,
  1618],
 'Færdiggørelsesbonus': [94, 133, 150, 151, 161, 154, 148, 135, 127, 124],
 'Udvekslingsstuderende': [13, 13, 14, 15, 16, 16, 15, 16, 16, 16],
 'Småfag': [12, 12, 11, 11, 11, 11, 11, 11, 11, 11],
 'Fripladser og stipendier': [None, None, None, 20, 18, 17, 14, 14, 14, 13],
 'Udvikling': [23, 21, 36, None, None, None, None, None, None, None],
 'Deltidsuddannelse': [22, 22, 22, 25, 22, 22, 22, 21, 21, 21],
 'Administrative effektivis

In [304]:
virk_df = pd.DataFrame(virk_dict)
optag_df = pd.DataFrame(optag_dict)
aktivitet_df = pd.DataFrame(aktivitet_dict)

In [306]:
virk_df

,Finanslov,År,I alt omsætning (mio kr),Tilskud i alt UI,Uddannelsestilskud fra UI,Heltidsuddannelse,Færdiggørelsesbonus,Udvekslingsstuderende,Småfag,Fripladser og stipendier,...,Basistilskud fra UI,Forskning,Kapitaltilskud,Forskningsbaseret myndighedsbetjening,Øvrige formål,Øvrige tilskud og indtægter i alt,Øvrige uddannelsestilskud mv,Tilskudsfinansieret forskning,Driftsindtægter,Andre tilskud
0,R,2007,6853,4669,1625,1461,94,13,12,NaN,...,3044,2322,73.0,5,640,2184,73,1179,819,108
1,R,2008,7132,4703,1640,1439,133,13,12,NaN,...,3063,2406,70.0,4,578,2429,80,1417,820,108
2,R,2009,7426,4936,1707,1474,150,14,11,NaN,...,3229,2585,46.0,4,589,2490,92,1582,703,109
3,R,2010,7777,4999,1790,1568,151,15,11,20.0,...,3209,2632,21.0,4,547,2778,84,1825,747,118
4,R,2011,7933,5097,1808,1580,161,16,11,18.0,...,3289,2792,NaN,5,475,2836,89,2009,648,85
5,B,2012,7826,4984,1789,1610,154,16,11,17.0,...,3195,2819,NaN,4,354,2842,100,2127,538,73
6,F,2013,7950,5035,1787,1660,148,15,11,14.0,...,3248,2876,NaN,13,350,2915,89,2209,532,72
7,BO 1,2014,8084,5057,1794,1680,135,16,11,14.0,...,3263,2855,NaN,13,386,3027,90,2320,532,72
8,BO 2,2015,8185,5043,1791,1685,127,16,11,14.0,...,3252,2842,NaN,13,388,3142,89,2436,532,72
9,BO 3,2016,8027,4763,1720,1618,124,16,11,13.0,...,3043,2631,NaN,13,391,3264,90,2557,532,72
